In [0]:
!pip install category_encoders
!pip install catboost
!pip install vincenty

In [0]:
import pandas as pd
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from sklearn.ensemble import BaggingClassifier

In [0]:
train = pd.read_csv("drive/My Drive/TrainZindi.csv")
test = pd.read_csv("drive/My Drive/TestZindi.csv")
Submission = pd.read_csv("drive/My Drive/SampleSubmissionZindi.csv")
riders = pd.read_csv("drive/My Drive/Riders.csv")
uber_data = pd.read_csv("drive/My Drive/Nairobi/Q1-2019.csv")
subloc = pd.read_json("drive/My Drive/nairobi_sublocations.json")
openrdatatrain = pd.read_csv("drive/My Drive/trainRoot.csv")
opdatatest = pd.read_csv("drive/My Drive/testRoot.csv")
uber_data_weekdays=pd.read_csv('drive/My Drive/Nairobi/uber_weekdays_hourly_q1.csv')
uber_data_weekends=pd.read_csv('drive/My Drive/Nairobi/uber_weekends_hourly_q1.csv')

In [0]:
train['ORIGIN_MOVEMENT_ID']=-1
train['DESTINATION_MOVEMENT_ID']=-1

In [0]:
for i in range(len(train)):
    pointpickup = Point(train.loc[i,'Pickup Long'],train.loc[i,'Pickup Lat'])
    pointdest = Point(train.loc[i,'Destination Long'],train.loc[i,'Destination Lat'])
    j = 0
    for j in range(len(subloc)):        
        if (Polygon(subloc['features'][j]['geometry']['coordinates'][0]).contains(pointpickup)):
            train.loc[i,'ORIGIN_MOVEMENT_ID'] = subloc.features[j]['properties']['MOVEMENT_ID']
            train.loc[i,'Origin_Place'] = subloc.features[j]['properties']['DISPLAY_NAME']
        if (Polygon(subloc['features'][j]['geometry']['coordinates'][0]).contains(pointdest)):
            train.loc[i,'DESTINATION_MOVEMENT_ID'] = subloc.features[j]['properties']['MOVEMENT_ID']
            train.loc[i,'Destination_Place'] = subloc.features[j]['properties']['DISPLAY_NAME']

In [0]:
test['ORIGIN_MOVEMENT_ID']=-1
test['DESTINATION_MOVEMENT_ID']=-1

In [0]:
for i in range(len(test)):
    pointpickup = Point(test.loc[i,'Pickup Long'],test.loc[i,'Pickup Lat'])
    pointdest = Point(test.loc[i,'Destination Long'],test.loc[i,'Destination Lat'])
    j = 0
    for j in range(len(subloc)):        
        if (Polygon(subloc['features'][j]['geometry']['coordinates'][0]).contains(pointpickup)):
            test.loc[i,'ORIGIN_MOVEMENT_ID'] = subloc.features[j]['properties']['MOVEMENT_ID']
            test.loc[i,'Origin_Place'] = subloc.features[j]['properties']['DISPLAY_NAME']
        if (Polygon(subloc['features'][j]['geometry']['coordinates'][0]).contains(pointdest)):
            test.loc[i,'DESTINATION_MOVEMENT_ID'] = subloc.features[j]['properties']['MOVEMENT_ID']
            test.loc[i,'Destination_Place'] = subloc.features[j]['properties']['DISPLAY_NAME']  
          

In [0]:
train.drop(['Vehicle Type'],axis=1,inplace=True)
train.columns

In [0]:
train = pd.merge(train,riders,on='Rider Id',how='left')
test = pd.merge(test,riders,on='Rider Id',how='left')
train['Time_from_Pickup_to_Arrival_Minutes'] = train['Time from Pickup to Arrival']/60
train['Speed_Meter_per_Second'] = (train['Distance (KM)']*1000)/train['Time from Pickup to Arrival']
OutliersTrain = train[(train.Speed_Meter_per_Second>=18)]#|(train.Speed_Meter_per_Second<=2)]
OutlierFreeTrain = train[~(train['Order No'].isin(OutliersTrain['Order No']))]
riders['WrongOrders'] = riders['Rider Id'].map(OutliersTrain.groupby('Rider Id').agg('count')['Order No'])
riders.WrongOrders.fillna(value=0,inplace=True)
train['Total_Orders'] = train['Rider Id'].map(train.groupby('Rider Id').agg('count')['Order No'])
train = pd.merge(train,riders.drop(['No_of_Ratings','Average_Rating','Age','No_Of_Orders'],axis=1),on='Rider Id',how='left')
test = pd.merge(test,riders.drop(['No_of_Ratings','Average_Rating','Age','No_Of_Orders'],axis=1),on='Rider Id',how='left')
train['Average_Rider_Speed'] = train['Rider Id'].map(OutlierFreeTrain.groupby('Rider Id').Speed_Meter_per_Second.mean())
test['Average_Rider_Speed'] =test['Rider Id'].map(OutlierFreeTrain.groupby('Rider Id').Speed_Meter_per_Second.mean())
train['error_rate_rider'] = train['WrongOrders']/train['Total_Orders']
test['WrongOrders'] = test['Rider Id'].map(riders.groupby('Rider Id').agg('sum').WrongOrders)
test['Total_Orders'] = test['Rider Id'].map(train.groupby('Rider Id').agg('count')['Order No'])
test['error_rate_rider'] = test.WrongOrders/test.Total_Orders
test.WrongOrders.fillna(value=train.WrongOrders.mean(),inplace=True)
test.error_rate_rider.fillna(value=train.error_rate_rider.mean(),inplace=True)
test.Total_Orders.fillna(value=0,inplace=True)
train.WrongOrders.fillna(value=train.WrongOrders.mean(),inplace=True)
train.error_rate_rider.fillna(value=train.error_rate_rider.mean(),inplace=True)
train.Total_Orders.fillna(value=0,inplace=True)
test.Average_Rider_Speed.fillna(value=OutlierFreeTrain.Speed_Meter_per_Second.mean(),inplace=True)

In [0]:
train['RatingFactor_Rider'] = train['Average_Rating']*train['No_of_Ratings']
test['RatingFactor_Rider'] = test['Average_Rating']*test['No_of_Ratings']
train['Productivity_Rider'] = train['Age']/train['No_Of_Orders']
test['Productivity_Rider'] = test['Age']/test['No_Of_Orders']

In [0]:
classifytrain = train.copy()
classifytest = test.copy()
classifytrain.rename({'Arrival at Destination - Day of Month':'Day_of_month','Arrival at Destination - Weekday (Mo = 1)':'Weekday'},axis=1,inplace=True)
classifytest.rename({'Arrival at Pickup - Day of Month':'Day_of_month','Arrival at Pickup - Weekday (Mo = 1)':'Weekday'},axis=1,inplace=True)
classifytrain['Confirmation - Time']=pd.to_datetime(classifytrain['Confirmation - Time'])
classifytrain['Placement - Time']=pd.to_datetime(classifytrain['Placement - Time'])
classifytrain['Pickup - Time'] = pd.to_datetime(classifytrain['Pickup - Time'])
classifytrain['Arrival at Pickup - Time'] = pd.to_datetime(classifytrain['Arrival at Pickup - Time'])
classifytest['Confirmation - Time']=pd.to_datetime(classifytest['Confirmation - Time'])
classifytest['Placement - Time']=pd.to_datetime(classifytest['Placement - Time'])
classifytest['Pickup - Time'] = pd.to_datetime(classifytest['Pickup - Time'])
classifytest['Arrival at Pickup - Time'] = pd.to_datetime(classifytest['Arrival at Pickup - Time'])
classifytrain['Placement_Time - Confirmation'] = (classifytrain['Confirmation - Time'] - classifytrain['Placement - Time']).astype('timedelta64[s]')
classifytrain['Confirmation - Arrival_Pickup'] = (classifytrain['Arrival at Pickup - Time']-classifytrain['Confirmation - Time']).astype('timedelta64[s]')
classifytrain['Arrival at Pickup - Pickup'] = (classifytrain['Pickup - Time']-classifytrain['Arrival at Pickup - Time']).astype('timedelta64[s]')
classifytest['Placement_Time - Confirmation'] = (classifytest['Confirmation - Time'] - classifytest['Placement - Time']).astype('timedelta64[s]')
classifytest['Confirmation - Arrival_Pickup'] = (classifytest['Arrival at Pickup - Time']-classifytest['Confirmation - Time']).astype('timedelta64[s]')
classifytest['Arrival at Pickup - Pickup'] = (classifytest['Pickup - Time']-classifytest['Arrival at Pickup - Time']).astype('timedelta64[s]')

In [0]:
## a tester avec classification et sans
train['Placement_Time - Confirmation'] = classifytrain['Placement_Time - Confirmation']
train['Confirmation - Arrival_Pickup'] = classifytrain['Confirmation - Arrival_Pickup']
train['Arrival at Pickup - Pickup'] = classifytrain['Arrival at Pickup - Pickup']
test['Placement_Time - Confirmation'] = classifytest['Placement_Time - Confirmation']
test['Confirmation - Arrival_Pickup'] = classifytest['Confirmation - Arrival_Pickup']
test['Arrival at Pickup - Pickup'] = classifytest['Arrival at Pickup - Pickup']

In [0]:
train['pos_pickup'] = train['Pickup Long'].apply(str)+'_'+train['Pickup Lat'].apply(str)
train['pos_dest'] = train['Destination Long'].apply(str)+'_'+train['Pickup Lat'].apply(str)
test['pos_pickup'] = test['Pickup Long'].apply(str)+'_'+test['Pickup Lat'].apply(str)
test['pos_dest'] = test['Destination Long'].apply(str)+'_'+test['Pickup Lat'].apply(str)
train['from_to']= train['pos_pickup']+'_'+train['pos_dest']
test['from_to']= test['pos_pickup']+'_'+test['pos_dest']

In [0]:
def not_weekend(x):
  if x<=5:
    return 1
  else :
    return 0
train["not_weekend"] = train["Placement - Weekday (Mo = 1)"].apply(not_weekend)
test["not_weekend"] = test["Placement - Weekday (Mo = 1)"].apply(not_weekend)

In [0]:
def quarter_of_month(x):
  if x>=1 and x<=7:
    return 1
  elif x>7 and x<=14:
    return 2
  elif x>14 and x<=21:
    return 3
  else:
    return 4
train["Placement - Day of Month_quarter_of_month"] = train["Placement - Day of Month"].apply(quarter_of_month)
test["Placement - Day of Month_quarter_of_month"] = test["Placement - Day of Month"].apply(quarter_of_month) 

In [0]:
train['Pickup - Time'] = pd.to_datetime(train['Pickup - Time'])
test['Pickup - Time'] = pd.to_datetime(test['Pickup - Time'])
train['Pickup_Hour'] = train['Pickup - Time'].dt.hour
test['Pickup_Hour'] = test['Pickup - Time'].dt.hour

In [0]:
train.loc[(train['Pickup_Hour']>=7)&(train['Pickup_Hour']<9),'DayTime']= 'Rush_Hour1'
train.loc[(train['Pickup_Hour']>=9)&(train['Pickup_Hour']<=11),'DayTime']= 'Matin'
train.loc[(train['Pickup_Hour']>11)&(train['Pickup_Hour']<=13),'DayTime']= '9ayla'
train.loc[(train['Pickup_Hour']>13)&(train['Pickup_Hour']<17),'DayTime']= 'ApresMidi'
train.loc[(train['Pickup_Hour']>=17)&(train['Pickup_Hour']<=19),'DayTime']= 'Rush_Hour2'
train.loc[(train['Pickup_Hour']>19)&(train['Pickup_Hour']<7),'DayTime']= 'Night'
test.loc[(test['Pickup_Hour']>=7)&(test['Pickup_Hour']<9),'DayTime']= 'Rush_Hour1'
test.loc[(test['Pickup_Hour']>=9)&(test['Pickup_Hour']<=11),'DayTime']= 'Matin'
test.loc[(test['Pickup_Hour']>11)&(test['Pickup_Hour']<=13),'DayTime']= '9ayla'
test.loc[(test['Pickup_Hour']>13)&(test['Pickup_Hour']<=16),'DayTime']= 'ApresMidi'
test.loc[(test['Pickup_Hour']>16)&(test['Pickup_Hour']<=21),'DayTime']= 'Night'
test.loc[(test['Pickup_Hour']>=17)&(test['Pickup_Hour']<=19),'DayTime']= 'Rush_Hour2'

In [0]:
train.rename({"ORIGIN_MOVEMENT_ID":"sourceid","DESTINATION_MOVEMENT_ID":"dstid","Pickup_Hour":"hod"},inplace=True,axis=1)
test.rename({"ORIGIN_MOVEMENT_ID":"sourceid","DESTINATION_MOVEMENT_ID":"dstid","Pickup_Hour":"hod"},inplace=True,axis=1)

In [0]:
train = pd.merge(train,uber_data, on=['sourceid', 'dstid','hod'],how="left")
test = pd.merge(test,uber_data, on=['sourceid', 'dstid','hod'],how="left")

In [0]:
data = [train, test]
for dataset in data:
    dataset.loc[ (dataset['hod'] < 13) , 'AM_PM'] = 'AM'
    dataset.loc[(dataset['hod'] >= 13) , 'AM_PM'] = 'PM'

In [0]:
train['Daytime_Day'] = train['DayTime']+'_'+train['Placement - Weekday (Mo = 1)'].apply(str)
test['Daytime_Day'] = test['DayTime']+'_'+test['Placement - Weekday (Mo = 1)'].apply(str)

In [0]:
from geopy.distance import vincenty
train["Minimum_distance_frompick_to_arrival"] = train.apply(lambda x: vincenty((x["Pickup Lat"], x["Pickup Long"]), (x["Destination Lat"], x["Destination Long"])).kilometers, axis = 1)
test["Minimum_distance_frompick_to_arrival"] = test.apply(lambda x: vincenty((x["Pickup Lat"], x["Pickup Long"]), (x["Destination Lat"], x["Destination Long"])).kilometers, axis = 1)
train["Distance-Minimum Distance"] = train["Distance (KM)"] - train["Minimum_distance_frompick_to_arrival"]
test["Distance-Minimum Distance"] = test["Distance (KM)"] - test["Minimum_distance_frompick_to_arrival"]
train.drop("Minimum_distance_frompick_to_arrival",axis=1,inplace=True)
test.drop("Minimum_distance_frompick_to_arrival",axis=1,inplace=True)

In [0]:
from vincenty import vincenty
JK = (-1.323053, 36.921824)
Wilson = (-1.321358, 36.815189)
Jogoo_Road = (-1.297140, 36.864817)
Langata_Road = (-1.312558, 36.816135)
Ngong_Road = (-1.335589, 36.672402)
Nairobi_Center =(-1.286389, 36.817223)
Parklands = (-1.261306, 36.816227)

In [0]:
train["Pickup_Distance_To_JK"] = train.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),JK),axis=1)
train["Pickup_Distance_To_Wilson"] = train.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Wilson),axis=1)
train["Pickup_Distance_To_Jogoo_Road"] = train.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Jogoo_Road),axis=1)
train["Pickup_Distance_To_Langata_Road"] = train.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Langata_Road),axis=1)
train["Pickup_Distance_To_Ngong_Road"] = train.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Ngong_Road),axis=1)
train["Pickup_Distance_To_Nairobi_Center"] = train.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Nairobi_Center),axis=1)
train["Pickup_Distance_To_Parklands"] = train.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Parklands),axis=1)
train["Destination_Distance_To_JK"] = train.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),JK),axis=1)
train["Destination_Distance_To_Wilson"] = train.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Wilson),axis=1)
train["Destination_Distance_To_Jogoo_Road"] = train.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Jogoo_Road),axis=1)
train["Destination_Distance_To_Langata_Road"] = train.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Langata_Road),axis=1)
train["Destination_Distance_To_Ngong_Road"] = train.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Ngong_Road),axis=1)
train["Destination_Distance_To_Nairobi_Center"] = train.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Nairobi_Center),axis=1)
train["Destination_Distance_To_Parklands"] = train.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Parklands),axis=1)


In [0]:
test["Pickup_Distance_To_JK"] = test.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),JK),axis=1)
test["Pickup_Distance_To_Wilson"] = test.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Wilson),axis=1)
test["Pickup_Distance_To_Jogoo_Road"] = test.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Jogoo_Road),axis=1)
test["Pickup_Distance_To_Langata_Road"] = test.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Langata_Road),axis=1)
test["Pickup_Distance_To_Ngong_Road"] = test.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Ngong_Road),axis=1)
test["Pickup_Distance_To_Nairobi_Center"] = test.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Nairobi_Center),axis=1)
test["Pickup_Distance_To_Parklands"] = test.apply(lambda row:vincenty((row["Pickup Lat"],row["Pickup Long"]),Parklands),axis=1)
test["Destination_Distance_To_JK"] = test.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),JK),axis=1)
test["Destination_Distance_To_Wilson"] = test.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Wilson),axis=1)
test["Destination_Distance_To_Jogoo_Road"] = test.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Jogoo_Road),axis=1)
test["Destination_Distance_To_Langata_Road"] = test.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Langata_Road),axis=1)
test["Destination_Distance_To_Ngong_Road"] = test.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Ngong_Road),axis=1)
test["Destination_Distance_To_Nairobi_Center"] = test.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Nairobi_Center),axis=1)
test["Destination_Distance_To_Parklands"] = test.apply(lambda row:vincenty((row["Destination Lat"],row["Destination Long"]),Parklands),axis=1)


In [0]:
train.loc[:, 'center_latitude'] = (train['Pickup Lat'].values + train['Destination Lat'].values) / 2
train.loc[:, 'center_longitude'] = (train['Pickup Long'].values + train['Destination Long'].values) / 2
test.loc[:, 'center_latitude'] = (test['Pickup Lat'].values + test['Destination Lat'].values) / 2
test.loc[:, 'center_longitude'] = (test['Pickup Long'].values + test['Destination Long'].values) / 2

In [0]:
from sklearn.decomposition import PCA
pca = PCA(random_state=50).fit(coords)
train['pickup_pca0'] = pca.transform(train[['Pickup Lat', 'Pickup Long']])[:, 0]
train['pickup_pca1'] = pca.transform(train[['Pickup Lat', 'Pickup Long']])[:, 1]
train['dropoff_pca0'] = pca.transform(train[['Destination Lat', 'Destination Long']])[:, 0]
train['dropoff_pca1'] = pca.transform(train[['Destination Lat', 'Destination Long']])[:, 1]
test['pickup_pca0'] = pca.transform(test[['Pickup Lat', 'Pickup Long']])[:, 0]
test['pickup_pca1'] = pca.transform(test[['Pickup Lat', 'Pickup Long']])[:, 1]
test['dropoff_pca0'] = pca.transform(test[['Destination Lat', 'Destination Long']])[:, 0]
test['dropoff_pca1'] = pca.transform(test[['Destination Lat', 'Destination Long']])[:, 1]

In [0]:
coords = np.vstack((train[['Pickup Lat', 'Pickup Long']].values,
                    train[['Destination Lat', 'Destination Long']].values))

In [0]:
from sklearn.cluster import MiniBatchKMeans
sample_ind = np.arange(start=0, stop=len(coords), step=1)
kmeans = MiniBatchKMeans(n_clusters=14, batch_size=11000,random_state=50).fit(coords[sample_ind])
train.loc[:, 'pickup_cluster'] = kmeans.predict(train[['Pickup Lat', 'Pickup Long']])
train.loc[:, 'dropoff_cluster'] = kmeans.predict(train[['Destination Lat', 'Destination Long']])
test.loc[:, 'pickup_cluster'] = kmeans.predict(test[['Pickup Lat', 'Pickup Long']])
test.loc[:, 'dropoff_cluster'] = kmeans.predict(test[['Destination Lat', 'Destination Long']])

In [0]:
def bearing_array(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

In [0]:
train.loc[:, 'direction'] = bearing_array(train['Pickup Lat'].values, train['Pickup Long'].values, train['Destination Lat'].values, train['Destination Long'].values)
test.loc[:, 'direction'] = bearing_array(test['Pickup Lat'].values, test['Pickup Long'].values, test['Destination Lat'].values, test['Destination Long'].values)


In [0]:
train["Rating13"] = train["Average_Rating"].apply(lambda x:1 if x>=13 and x<14 else 0)
train["Rating14"] = train["Average_Rating"].apply(lambda x:1 if x>=14 and x<15 else 0)
train["Rating15+"] = train["Average_Rating"].apply(lambda x:1 if x>=15 else 0)
train["Rating_Under_11"] = train["Average_Rating"].apply(lambda x:1 if x<11 else 0)
train["Rating_between_11_12"] = train["Average_Rating"].apply(lambda x:1 if x>=11 and x<13 else 0)

test["Rating13"] = test["Average_Rating"].apply(lambda x:1 if x>=13 and x<14 else 0)
test["Rating14"] = test["Average_Rating"].apply(lambda x:1 if x>=14 and x<15 else 0)
test["Rating15+"] = test["Average_Rating"].apply(lambda x:1 if x>=15 else 0)
test["Rating_Under_11"] = test["Average_Rating"].apply(lambda x:1 if x<11 else 0)
test["Rating_between_11_12"] = test["Average_Rating"].apply(lambda x:1 if x>=11 and x<13 else 0)





In [0]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def dummy_manhattan_distance(lat1, lng1, lat2, lng2):
    a = haversine_array(lat1, lng1, lat1, lng2)
    b = haversine_array(lat1, lng1, lat2, lng1)
    return a + b
train.loc[:, 'distance_haversine'] = haversine_array(train['Pickup Lat'].values, train['Pickup Long'].values, train['Destination Lat'].values, train['Destination Long'].values)
test.loc[:, 'distance_haversine'] = haversine_array(test['Pickup Lat'].values, test['Pickup Long'].values, test['Destination Lat'].values, test['Destination Long'].values)


In [0]:
train.loc[:, 'distance_manhattan'] = dummy_manhattan_distance(train['Pickup Lat'].values, train['Pickup Long'].values, train['Destination Lat'].values, train['Destination Long'].values)
test.loc[:, 'distance_manhattan'] = dummy_manhattan_distance(test['Pickup Lat'].values, test['Pickup Long'].values, test['Destination Lat'].values, test['Destination Long'].values)

In [0]:
train.loc[:, 'pickup_lat_bin'] = np.round(train['Pickup Lat'], 2)
train.loc[:, 'pickup_long_bin'] = np.round(train['Pickup Long'], 2)
train.loc[:, 'dest_lat_bin'] = np.round(train['Destination Lat'],2)
train.loc[:, 'dest_long_bin'] = np.round(train['Destination Long'], 2)
test.loc[:, 'pickup_lat_bin'] = np.round(test['Pickup Lat'], 2)
test.loc[:, 'pickup_long_bin'] = np.round(test['Pickup Long'], 2)
test.loc[:, 'dest_lat_bin'] = np.round(test['Destination Lat'], 2)
test.loc[:, 'dest_long_bin'] = np.round(test['Destination Long'], 2)

In [0]:
train['Pickup - Time'] = pd.to_datetime(train['Pickup - Time'])
test['Pickup - Time'] = pd.to_datetime(test['Pickup - Time'])
train['Pickup_Hour'] = train['Pickup - Time'].dt.hour
test['Pickup_Hour'] = test['Pickup - Time'].dt.hour

In [0]:
train['Confirmation - Time'] = pd.to_datetime(train['Confirmation - Time'])
test['Confirmation - Time'] = pd.to_datetime(test['Confirmation - Time'])
train['Confirmation_Hour'] = train['Confirmation - Time'].dt.hour
test['Confirmation_Hour'] = test['Confirmation - Time'].dt.hour

In [0]:
hours_in_day = 24

train['sin_hour_pickup'] = np.sin(2*np.pi*train.Pickup_Hour/hours_in_day)
train['cos_hour_pickup'] = np.cos(2*np.pi*train.Pickup_Hour/hours_in_day)
test['sin_hour_pickup'] = np.sin(2*np.pi*test.Pickup_Hour/hours_in_day)
test['cos_hour_pickup'] = np.cos(2*np.pi*test.Pickup_Hour/hours_in_day)

In [0]:
hours_in_day = 24

train['sin_hour_confirmation'] = np.sin(2*np.pi*train.Pickup_Hour/hours_in_day)
train['cos_hour_confirmation'] = np.cos(2*np.pi*train.Pickup_Hour/hours_in_day)
test['sin_hour_confirmation'] = np.sin(2*np.pi*test.Pickup_Hour/hours_in_day)
test['cos_hour_confirmation'] = np.cos(2*np.pi*test.Pickup_Hour/hours_in_day)

In [0]:
most_congested_locations = ["Spring Valley","Upper Parklands","Landmawe","Kilimani","City Square","Muthangari","Woodley","Highridge","South 'c'","Kileleshwa","Maziwa",'Kenyatta']

In [0]:
for location in most_congested_locations:
  train['Departure'+'_'+location] = train["Origin_Place"].apply(lambda x:1 if x==location else 0)
  train['Arrival'+'_'+location] = train["Destination_Place"].apply(lambda x:1 if x==location else 0)
  test['Departure'+'_'+location] = test["Origin_Place"].apply(lambda x:1 if x==location else 0)
  test['Arrival'+'_'+location] = test["Destination_Place"].apply(lambda x:1 if x==location else 0)


In [0]:
for i in range(len(train)):
  print(i)
  for j in range(len(subloc)):
    longitude = list(Polygon(subloc['features'][j]['geometry']['coordinates'][0]).centroid.coords)[0][0]
    latitude = list(Polygon(subloc['features'][j]['geometry']['coordinates'][0]).centroid.coords)[0][1]
    train.loc[i,"distance_Departure_from_poly_{}".format(j)] = vincenty((latitude,longitude),(train.iloc[i]["Pickup Lat"],train.iloc[i]["Pickup Long"]))
    train.loc[i,"distance_Arrival_from_poly_{}".format(j)] = vincenty((latitude,longitude),(train.iloc[i]["Destination Lat"],train.iloc[i]["Destination Long"]))


In [0]:
for i in range(len(test)):
  print(i)
  for j in range(len(subloc)):
    longitude = list(Polygon(subloc['features'][j]['geometry']['coordinates'][0]).centroid.coords)[0][0]
    latitude = list(Polygon(subloc['features'][j]['geometry']['coordinates'][0]).centroid.coords)[0][1]
    test.loc[i,"distance_Departure_from_poly_{}".format(j)] = vincenty((latitude,longitude),(test.iloc[i]["Pickup Lat"],test.iloc[i]["Pickup Long"]))
    test.loc[i,"distance_Arrival_from_poly_{}".format(j)] = vincenty((latitude,longitude),(test.iloc[i]["Destination Lat"],test.iloc[i]["Destination Long"]))


In [0]:
test.tail()

In [0]:
def haversine_dist(lat1,lng1,lat2,lng2):
  lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
  radius = 6371  # Earth's radius taken from google
  lat = lat2 - lat1
  lng = lng2 - lng1
  d = np.sin(lat/2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng/2) ** 2
  h = 2 * radius * np.arcsin(np.sqrt(d))
  return h

In [0]:
train.drop(['mean_travel_time','standard_deviation_travel_time'],axis=1,inplace=True)
test.drop(['mean_travel_time','standard_deviation_travel_time'],axis=1,inplace=True)

In [0]:
error_mean=train.error_rate_rider.mean()
train.loc[(train.error_rate_rider>=0.2)&(train.Total_Orders<=5),'error_rate_rider']=error_mean
test.loc[(test.error_rate_rider>=0.2)&(test.Total_Orders<=5),'error_rate_rider']=error_mean

In [0]:
train = pd.merge(train,uber_data_weekdays, on=['sourceid', 'dstid','hod'],how="left")
test = pd.merge(test,uber_data_weekdays, on=['sourceid', 'dstid','hod'],how="left")

In [0]:
train.rename({'mean_travel_time':'mean_travel_time_y','standard_deviation_travel_time':'standard_deviation_travel_time_y'},axis=1,inplace=True)
test.rename({'mean_travel_time':'mean_travel_time_y','standard_deviation_travel_time':'standard_deviation_travel_time_y'},axis=1,inplace=True)

In [0]:
train.loc[train.not_weekend==0,'mean_travel_time_y']=-1
train.loc[train.not_weekend==0,'standard_deviation_travel_time_y']=-1
test.loc[test.not_weekend==0,'mean_travel_time_y']=-1
test.loc[test.not_weekend==0,'standard_deviation_travel_time_y']=-1

In [0]:
train = pd.merge(train,uber_data_weekends, on=['sourceid', 'dstid','hod'],how="left")
test = pd.merge(test,uber_data_weekends, on=['sourceid', 'dstid','hod'],how="left")

In [0]:
#train.loc[train.mean_travel_time_y==-1,'mean_travel_time_y']=train.mean_travel_time
#train.loc[train.standard_deviation_travel_time_y==-1,'standard_deviation_travel_time_y']=train.standard_deviation_travel_time
#test.loc[test.mean_travel_time_y==-1,'mean_travel_time_y']=test.mean_travel_time
#test.loc[test.standard_deviation_travel_time_y==-1,'standard_deviation_travel_time_y']=test.standard_deviation_travel_time

In [0]:
train.loc[train.mean_travel_time_y==-1,'mean_travel_time_y']=train.loc[train.mean_travel_time_y==-1,'mean_travel_time']
train.loc[train.standard_deviation_travel_time_y==-1,'standard_deviation_travel_time_y']=train.loc[train.standard_deviation_travel_time_y==-1,'standard_deviation_travel_time']
test.loc[test.mean_travel_time_y==-1,'mean_travel_time_y']=test.loc[test.mean_travel_time_y==-1,'mean_travel_time']
test.loc[test.standard_deviation_travel_time_y==-1,'standard_deviation_travel_time_y']=test.loc[test.standard_deviation_travel_time_y==-1,'standard_deviation_travel_time']

In [0]:
train=train[train['Time from Pickup to Arrival']<=5000]

In [0]:
features1 = ['Platform Type','Personal or Business', #'Total_Population',
        'not_weekend','Placement - Day of Month_quarter_of_month',
       'distance_haversine',#'Daytime_Day', 
             #'Temperature',
        'Rider Id','DayTime',#'AM_PM',# 'pos_pickup','pos_dest',
             #'cluster_to_cluster',#'from_to',
             'Destination_Distance_To_Nairobi_Center','Pickup_Distance_To_Nairobi_Center',
       'Pickup_Distance_To_Wilson','Pickup_Distance_To_Jogoo_Road',
             'Pickup_Distance_To_Langata_Road','Pickup_Distance_To_Ngong_Road',
      'Destination_Distance_To_Wilson','Destination_Distance_To_Jogoo_Road',
           'Destination_Distance_To_Langata_Road', 'Destination_Distance_To_Ngong_Road',
             'Pickup_Distance_To_JK','Destination_Distance_To_JK',
             #'Pickup_Distance_To_Parklands',#'Destination_Distance_To_Parklands',
        #'User Id',
        #'No_Of_Orders', 'Age', 'Average_Rating', 'No_of_Ratings',
       #'WrongOrders',  
        'Pickup Lat','Pickup Long','Destination Lat','Destination Long' ,    
       'error_rate_rider','RatingFactor_Rider', 'Productivity_Rider','Average_Rider_Speed',
        'Placement_Time - Confirmation', 'Confirmation - Arrival_Pickup',
       'Arrival at Pickup - Pickup','Distance-Minimum Distance',
             'pickup_pca0', 'pickup_pca1','dropoff_pca0', 'dropoff_pca1',
       'center_latitude', 'center_longitude','pickup_cluster', 'dropoff_cluster','direction',
        'Rating13', 'Rating14', 'Rating15+', 'Rating_Under_11', 'Rating_between_11_12',
             #'mean_travel_time_x',
             'mean_travel_time_y','standard_deviation_travel_time_y',
 'cos_hour_pickup','sin_hour_pickup',
  'openroute_duration',
             #'standard_deviation_travel_time_x'
              'distance_Departure_from_poly_1',
 'distance_Arrival_from_poly_1',
  'distance_Departure_from_poly_2',
 'distance_Arrival_from_poly_2',
  'distance_Departure_from_poly_3',
 'distance_Arrival_from_poly_3',
  'distance_Departure_from_poly_4',
 'distance_Arrival_from_poly_4',
 
 'distance_Departure_from_poly_21',
 'distance_Arrival_from_poly_21',
 'distance_Departure_from_poly_22',
 'distance_Arrival_from_poly_22',
 'distance_Departure_from_poly_23',
 'distance_Arrival_from_poly_23',
 'distance_Departure_from_poly_24',
 'distance_Arrival_from_poly_24',
 'distance_Departure_from_poly_25',
 'distance_Arrival_from_poly_25',
 'distance_Departure_from_poly_26',
 'distance_Arrival_from_poly_26',
 'distance_Departure_from_poly_27',
 'distance_Arrival_from_poly_27',
 'distance_Departure_from_poly_28',
 'distance_Arrival_from_poly_28',



 'distance_Departure_from_poly_50',
 'distance_Arrival_from_poly_50',
 'distance_Departure_from_poly_51',
 'distance_Arrival_from_poly_51',
 'distance_Departure_from_poly_52',
 'distance_Arrival_from_poly_52',
 'distance_Departure_from_poly_53',
 'distance_Arrival_from_poly_53',
 'distance_Departure_from_poly_54',
 'distance_Arrival_from_poly_54',
 'distance_Departure_from_poly_55',
 'distance_Arrival_from_poly_55',
 'distance_Departure_from_poly_56',
 'distance_Arrival_from_poly_56',
 'distance_Departure_from_poly_57',
 'distance_Arrival_from_poly_57',
 'distance_Departure_from_poly_58',
 'distance_Arrival_from_poly_58',
 'distance_Departure_from_poly_59',
 'distance_Arrival_from_poly_59',
 'distance_Departure_from_poly_60',
 'distance_Arrival_from_poly_60',
 'distance_Departure_from_poly_61',
 'distance_Arrival_from_poly_61',
 'distance_Departure_from_poly_62',
 'distance_Arrival_from_poly_62',
 'distance_Departure_from_poly_63',
 'distance_Arrival_from_poly_63',
 'distance_Departure_from_poly_64',
 'distance_Arrival_from_poly_64',
 'distance_Departure_from_poly_65',
 'distance_Arrival_from_poly_65',
 'distance_Departure_from_poly_66',
 'distance_Arrival_from_poly_66',
 'distance_Departure_from_poly_67',
 'distance_Arrival_from_poly_67',
 'distance_Departure_from_poly_68',
 'distance_Arrival_from_poly_68',
 'distance_Departure_from_poly_69',
 'distance_Arrival_from_poly_69',
 'distance_Departure_from_poly_70',
 'distance_Arrival_from_poly_70',
 'distance_Departure_from_poly_71',
 'distance_Arrival_from_poly_71',
 'distance_Departure_from_poly_72',
 'distance_Arrival_from_poly_72',
 'distance_Departure_from_poly_73',
 'distance_Arrival_from_poly_73',
 'distance_Departure_from_poly_74',
 'distance_Arrival_from_poly_74',
 'distance_Departure_from_poly_75',
 'distance_Arrival_from_poly_75',
 'distance_Departure_from_poly_76',
 'distance_Arrival_from_poly_76',
 'distance_Departure_from_poly_77',
 'distance_Arrival_from_poly_77',
 'distance_Departure_from_poly_78',
 'distance_Arrival_from_poly_78',
 'distance_Departure_from_poly_79',
 'distance_Arrival_from_poly_79',

 'distance_Departure_from_poly_93',
 'distance_Arrival_from_poly_93',
 'distance_Departure_from_poly_94',
 'distance_Arrival_from_poly_94',
 'distance_Departure_from_poly_95',
 'distance_Arrival_from_poly_95',
 'distance_Departure_from_poly_96',
 'distance_Arrival_from_poly_96',
 'distance_Departure_from_poly_97',
 'distance_Arrival_from_poly_97',
 'distance_Departure_from_poly_98',
 'distance_Arrival_from_poly_98',
 'distance_Departure_from_poly_99',
 'distance_Arrival_from_poly_99',
 'distance_Departure_from_poly_100',
 'distance_Arrival_from_poly_100',
 'distance_Departure_from_poly_101',
 'distance_Arrival_from_poly_101',
 'distance_Departure_from_poly_102',
 'distance_Arrival_from_poly_102',
 'distance_Departure_from_poly_103',
 'distance_Arrival_from_poly_103',
 'distance_Departure_from_poly_104',
 'distance_Arrival_from_poly_104',
 'distance_Departure_from_poly_106',
 'distance_Arrival_from_poly_106']

In [0]:
oce = ce.OneHotEncoder(cols=['Platform Type','Personal or Business','DayTime'])#,'AM_PM'])#,'Daytime_Day'])
hce = ce.TargetEncoder(cols=['Rider Id'],smoothing=40,min_samples_leaf=7)
X = train[features1]
y = train['Time from Pickup to Arrival']
X_test = test[features1]
X = oce.fit_transform(X)
X = hce.fit_transform(X,y)
X_test = oce.transform(X_test)
X_test = hce.transform(X_test)
X.Average_Rider_Speed.fillna(value=X["Average_Rider_Speed"].mean(),inplace=True)
X_test.Average_Rider_Speed.fillna(value=X["Average_Rider_Speed"].mean(),inplace=True)
X.mean_travel_time_y.fillna(value=X["mean_travel_time_y"].mean(),inplace=True)
X_test.mean_travel_time_y.fillna(value=X["mean_travel_time_y"].mean(),inplace=True)
X.standard_deviation_travel_time_y.fillna(value=X["standard_deviation_travel_time_y"].mean(),inplace=True)
X_test.standard_deviation_travel_time_y.fillna(value=X["standard_deviation_travel_time_y"].mean(),inplace=True)

#X.geometric_standard_deviation_travel_time_y.fillna(value=X["geometric_standard_deviation_travel_time_y"].mean(),inplace=True)
#X_test.geometric_standard_deviation_travel_time_y.fillna(value=X_test["geometric_standard_deviation_travel_time_y"].mean(),inplace=True)
#X.geometric_mean_travel_time_y.fillna(value=X["geometric_mean_travel_time_y"].mean(),inplace=True)
#X_test.geometric_mean_travel_time_y.fillna(value=X_test["geometric_mean_travel_time_y"].mean(),inplace=True)
#X.speed_kph_p85.fillna(value=X["speed_kph_p85"].mean(),inplace=True)
#X_test.speed_kph_p85.fillna(value=X_test["speed_kph_p85"].mean(),inplace=True)

In [0]:
nfolds = 10
kfolds=KFold(n_splits=nfolds, shuffle=True, random_state=10)

In [0]:
oof_train_2 = np.zeros((len(X)))
oof_test_2 = np.zeros((len(test)))
cv_scores_2 = []
train_scores_2=[]
r2_train_scores_2=[]
test_scores_2=[]
r2_cv_scores_2=[]
for ind,(ind_train,ind_val) in (enumerate (kfolds.split(X,y))): 
    X_train,X_val = X.iloc[ind_train],X.iloc[ind_val] 
    y_train,y_val = y.iloc[ind_train],y.iloc[ind_val]
    #model=CatBoostRegressor(cat_features=cat_features)
    model=BaggingRegressor(CatBoostRegressor(max_depth=5,learning_rate=0.18,n_estimators=422,colsample_bylevel=0.6,loss_function='RMSE',random_state=50,l2_leaf_reg=2,subsample=0.9),random_state=50)
    model.fit(X_train,y_train)
    val_pred = model.predict(X_val)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test) # predict par fold sur le test
    oof_test_2 += test_pred # contient pour chaque fold des prédictions pour tout le test
    oof_train_2[ind_val] += val_pred
    score_fold_validation_2=np.sqrt(mean_squared_error(y_val, val_pred))
    score_fold_train_2=np.sqrt(mean_squared_error(y_train, train_pred))
    train_scores_2.append(score_fold_train_2)
    cv_scores_2.append(score_fold_validation_2)
    print('Iteration : {} - Train XGB Score : {} - CV XGB Score : {}'.format(str(ind+1),r2_score(y_train,train_pred),r2_score(y_val,val_pred)))
    r2_train_scores_2.append(r2_score(y_train,train_pred))
    r2_cv_scores_2.append(r2_score(y_val,val_pred))
end_train_score_2=np.mean(train_scores_2)
train_scores_2.append(end_train_score_2)
end_cv_score_2=np.mean(cv_scores_2)
cv_scores_2.append(end_cv_score_2)
end_r2_train_scores_2=np.mean(r2_train_scores_2)
end_r2_cv_scores_2 = np.mean(r2_cv_scores_2)
print('End_train_score : {} -  End_cv_score : {} '.format(end_train_score_2,end_cv_score_2))
print('End_R²_train : {} -  End_R²_cv : {} '.format(end_r2_train_scores_2,end_r2_cv_scores_2))

In [0]:
from sklearn.ensemble import BaggingRegressor

In [0]:
oof_train = np.zeros((len(X)))
oof_test = np.zeros((len(test)))
cv_scores = []
train_scores=[]
r2_train_scores=[]
test_scores=[]
r2_cv_scores=[]
for ind,(ind_train,ind_val) in (enumerate (kfolds.split(X,y))): 
    X_train,X_val = X.iloc[ind_train],X.iloc[ind_val] 
    y_train,y_val = y.iloc[ind_train],y.iloc[ind_val]
    #model=CatBoostRegressor(cat_features=cat_features)
    model=XGBRegressor(max_depth=4,learning_rate=0.17,n_estimators=422,min_child_weight =80,colsample_bytree=0.5,random_state=50,colsample_bylevel=0.6,objective='reg:squarederror')#,colsample_bynode=0.6)
    model.fit(X_train,y_train)
    val_pred = model.predict(X_val)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test) # predict par fold sur le test
    oof_test += test_pred # contient pour chaque fold des prédictions pour tout le test
    oof_train[ind_val] += val_pred
    score_fold_validation=np.sqrt(mean_squared_error(y_val, val_pred))
    score_fold_train=np.sqrt(mean_squared_error(y_train, train_pred))
    train_scores.append(score_fold_train)
    cv_scores.append(score_fold_validation)
    print('Iteration : {} - Train XGB Score : {} - CV XGB Score : {}'.format(str(ind+1),r2_score(y_train,train_pred),r2_score(y_val,val_pred)))
    r2_train_scores.append(r2_score(y_train,train_pred))
    r2_cv_scores.append(r2_score(y_val,val_pred))
end_train_score=np.mean(train_scores)
train_scores.append(end_train_score)
end_cv_score=np.mean(cv_scores)
cv_scores.append(end_cv_score)

end_r2_train_scores=np.mean(r2_train_scores)
end_r2_cv_scores = np.mean(r2_cv_scores)
print('End_train_score : {} -  End_cv_score : {} '.format(end_train_score,end_cv_score))
print('End_R²_train : {} -  End_R²_cv : {} '.format(end_r2_train_scores,end_r2_cv_scores))

In [0]:
!pip install lightgbm


In [0]:
from lightgbm import LGBMRegressor


In [0]:
oof_train_4 = np.zeros((len(X)))
oof_test_4 = np.zeros((len(test)))
cv_scores_4 = []
train_scores_4=[]
r2_train_scores_4=[]
test_scores_4=[]
r2_cv_scores_4=[]
for ind,(ind_train,ind_val) in (enumerate (kfolds.split(X,y))): 
    X_train,X_val = X.iloc[ind_train],X.iloc[ind_val] 
    y_train,y_val = y.iloc[ind_train],y.iloc[ind_val]
    #model=CatBoostRegressor(cat_features=cat_features)
    model = LGBMRegressor(max_depth=4,learning_rate=0.17,n_estimators=420,min_child_weight =80,colsample_bytree=0.5,random_state=50,colsample_bylevel=0.6)
    model.fit(X_train,y_train)
    val_pred = model.predict(X_val)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test) # predict par fold sur le test
    oof_test_4 += test_pred # contient pour chaque fold des prédictions pour tout le test
    oof_train_4[ind_val] += val_pred
    score_fold_validation_4=np.sqrt(mean_squared_error(y_val, val_pred))
    score_fold_train_4=np.sqrt(mean_squared_error(y_train, train_pred))
    train_scores_4.append(score_fold_train_4)
    cv_scores_4.append(score_fold_validation_4)
    print('Iteration : {} - Train XGB Score : {} - CV XGB Score : {}'.format(str(ind+1),r2_score(y_train,train_pred),r2_score(y_val,val_pred)))
    r2_train_scores_4.append(r2_score(y_train,train_pred))
    r2_cv_scores_4.append(r2_score(y_val,val_pred))
end_train_score_4=np.mean(train_scores_4)
train_scores_4.append(end_train_score_4)
end_cv_score_4=np.mean(cv_scores_4)
cv_scores_4.append(end_cv_score_4)
end_r2_train_scores_4=np.mean(r2_train_scores_4)
end_r2_cv_scores_4 = np.mean(r2_cv_scores_4)
print('End_train_score : {} -  End_cv_score : {} '.format(end_train_score_4,end_cv_score_4))
print('End_R²_train : {} -  End_R²_cv : {} '.format(end_r2_train_scores_4,end_r2_cv_scores_4))

In [0]:
from sklearn.ensemble import GradientBoostingRegressor

In [0]:
oof_train_3 = np.zeros((len(X)))
oof_test_3 = np.zeros((len(test)))
cv_scores_3 = []
train_scores_3=[]
r2_train_scores_3=[]
test_scores_3=[]
r2_cv_scores_3=[]
for ind,(ind_train,ind_val) in (enumerate (kfolds.split(X,y))): 
    X_train,X_val = X.iloc[ind_train],X.iloc[ind_val] 
    y_train,y_val = y.iloc[ind_train],y.iloc[ind_val]
    #model=CatBoostRegressor(cat_features=cat_features)
    model= GradientBoostingRegressor(n_estimators=420, learning_rate=0.17,
                                   max_depth=5, max_features='auto',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =42)
    model.fit(X_train,y_train)
    val_pred = model.predict(X_val)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test) # predict par fold sur le test
    oof_test_3 += test_pred # contient pour chaque fold des prédictions pour tout le test
    oof_train_3[ind_val] += val_pred
    score_fold_validation_3=np.sqrt(mean_squared_error(y_val, val_pred))
    score_fold_train_3=np.sqrt(mean_squared_error(y_train, train_pred))
    train_scores_3.append(score_fold_train_3)
    cv_scores_3.append(score_fold_validation_3)
    print('Iteration : {} - Train XGB Score : {} - CV XGB Score : {}'.format(str(ind+1),r2_score(y_train,train_pred),r2_score(y_val,val_pred)))
    r2_train_scores_3.append(r2_score(y_train,train_pred))
    r2_cv_scores_3.append(r2_score(y_val,val_pred))
end_train_score_3=np.mean(train_scores_3)
train_scores_3.append(end_train_score_3)
end_cv_score_3=np.mean(cv_scores_3)
cv_scores_3.append(end_cv_score_3)
end_r2_train_scores_3=np.mean(r2_train_scores_3)
end_r2_cv_scores_3 = np.mean(r2_cv_scores_3)
print('End_train_score : {} -  End_cv_score : {} '.format(end_train_score_3,end_cv_score_3))
print('End_R²_train : {} -  End_R²_cv : {} '.format(end_r2_train_scores_3,end_r2_cv_scores_3))

In [0]:
submissionoof = pd.DataFrame(index=test.index,columns=['Order No','Time from Pickup to Arrival'])
submissionoof['Order No'] = test['Order No']
submissionoof['Time from Pickup to Arrival'] = (oof_test/10)*0.00+(oof_test_2/10)*0.9+(oof_test_3/10)*0.1 + (oof_test_4/10)*0.00

In [0]:
plt.figure(figsize=(15,8))
plt.scatter(train['Time from Pickup to Arrival'],oof_train_2+oof_train+oof_train_3+oof_train_4)

In [0]:
cvcv=pd.DataFrame(columns=['Order No','Time from Pickup to Arrival'])
cvcv['Time from Pickup to Arrival'] =  oof_train*0.1+oof_train_2*0.7+ oof_train_3*0.1+ oof_train_4*0.1
print(np.sqrt(mean_squared_error(y, cvcv['Time from Pickup to Arrival'])))

In [0]:
train[(train.index.isin(cvcv.loc[cvcv['Time from Pickup to Arrival']<=1].index))]

In [0]:
cvcv.loc[cvcv['Time from Pickup to Arrival']<=1]

In [0]:
cvcv.loc[cvcv['Time from Pickup to Arrival']<=1,'Time from Pickup to Arrival']=2

In [0]:
print(np.sqrt(mean_squared_error(y, cvcv['Time from Pickup to Arrival'])))

In [0]:
submissionoof[submissionoof['Time from Pickup to Arrival']>4400]

In [0]:
submissionoof.loc[submissionoof['Time from Pickup to Arrival']<=1,'Time from Pickup to Arrival']=1

In [0]:
submissionoof.to_csv('drive/My Drive/Submission.csv',index=False)

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns



corrmat = X.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(250,250))
#plot heat map
g=sns.heatmap(X[top_corr_features].corr(),annot=True,cmap="RdYlGn")